## Jak działa partycjonowanie

1. Rozpocznij z 8 partycjami.
2. Uruchom kod.
3. Otwórz **Spark UI**
4. Sprawdź drugi job (czy są jakieś różnice pomięczy drugim)
5. Sprawdź **Event Timeline**
6. Sprawdzaj czas wykonania.
  * Uruchom kilka razy rzeby sprawdzić średni czas wykonania.

Powtórz z inną liczbą partycji
* 1 partycja
* 7 partycja
* 9 partycja
* 16 partycja
* 24 partycja
* 96 partycja
* 200 partycja
* 4000 partycja

Zastąp `repartition(n)` z `coalesce(n)` używając:
* 6 partycji
* 5 partycji
* 4 partycji
* 3 partycji
* 2 partycji
* 1 partycji

** *Note:* ** *Dane muszą być wystarczająco duże żeby zaobserwować duże różnice z małymi partycjami.*<br/>* To co możesz sprawdzić jak zachowują się małe dane z dużą ilośćia partycji.*

In [0]:
# slots = sc.defaultParallelism
spark.conf.get("spark.sql.shuffle.partitions")

Out[1]: '200'

In [0]:
spark.catalog.clearCache()
parquetDir = "/FileStore/tables/training/wikipedia/pageviews/"

df = (spark.read
  .parquet(parquetDir)
.repartition(2000)
#    .coalesce(6)
.groupBy("count_views").sum())


df.explain
df.count()

Out[2]: 1133

In [0]:
df.rdd.getNumPartitions

Out[3]: <bound method RDD.getNumPartitions of MapPartitionsRDD[130] at javaToPython at NativeMethodAccessorImpl.java:0>

In [0]:
import time

for n in [1, 7, 9, 16, 24, 96, 200, 4000]:
    start = time.time()
    df = df.repartition(n)
    df.write.mode("overwrite").format("noop").save()
    print("Czas wykonania:", time.time() - start)


Czas wykonania: 72.98877906799316
Czas wykonania: 190.10655570030212
Czas wykonania: 194.7217893600464
Czas wykonania: 194.4560730457306
Czas wykonania: 191.49872970581055
Czas wykonania: 206.26620197296143
Czas wykonania: 202.04097819328308
Czas wykonania: 449.8258807659149


In [0]:
for n in [6, 5, 4, 3, 2, 1]:
    start = time.time()
    df_small = df.coalesce(n)
    df_small.write.mode("overwrite").format("noop").save()
    end = time.time()
    print(f"Coalesce({n}) - Czas wykonania: {round(end - start, 2)} sek")

Coalesce(6) - Czas wykonania: 225.24 sek
Coalesce(5) - Czas wykonania: 345.5 sek
Coalesce(4) - Czas wykonania: 258.71 sek
Coalesce(3) - Czas wykonania: 253.49 sek
Coalesce(2) - Czas wykonania: 255.75 sek
Coalesce(1) - Czas wykonania: 281.44 sek
